# Ecological Metadata Language (EML)

Some archives include additional metadata stored in the form of an XML file that uses [Ecological Metadata Language (EML)](https://eml.ecoinformatics.org/).  This path to this file can be retrieved from the `ArchiveReader.MetaData`.  This notebook describes how information can be retrieved from the dataset metadata and presented.

In [1]:
#r "nuget:DwC-A_dotnet,0.6.0"
#r "nuget:DwC-A_dotnet.Interactive,0.1.8-Pre"

In [1]:
using DwC_A;
using System.IO;

var archive = new ArchiveReader("./data/dwca-rooftop-v1.4.zip");
var emlPath = Path.Combine(archive.OutputPath, archive.MetaData.Metadata);
emlPath

## Using XPath Queries

Using an XPathNavigator object we can query into any part of the document and retrieve single elements or sections.

For some tips on finding the information you need see the document [Best Practices for Dataset Metadata in Ecological Metadata Language (EML)](https://environmentaldatainitiative.org/five-phases-of-data-publishing/phase-3/metadata-best-practices/) available for download from [https://environmentaldatainitiative.org](https://environmentaldatainitiative.org).

In [1]:
using System.Xml;
using System.Xml.XPath;

var docNav = new XPathDocument(emlPath);
var nav = docNav.CreateNavigator();
XmlNamespaceManager nsmgr = new XmlNamespaceManager(nav.NameTable);
nsmgr.AddNamespace("eml", "eml://ecoinformatics.org/eml-2.1.1");
var root = nav.SelectSingleNode("eml:eml", nsmgr);

var title = root.SelectSingleNode("dataset/title", nsmgr).ToString();
title

## Formatter Shortcut

The following Formatters can be used as a shortcut to display the results of simple queries.  Try some of the following in the cell following the next.

* dataset/title
* dataset/abstract/para
* dataset/keywordSet/keyword
* additionalMetadata/metadata/gbif/citation

In [1]:
using Microsoft.DotNet.Interactive.Formatting;
using System.Xml.XPath;

using static  Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

Formatter.Register<XPathNavigator>((nav, writer) => {
    var output = p(nav.ToString());
    writer.Write(output);
}, "text/html");

Formatter.Register<XPathNodeIterator>((iter, writer) => {
    var outputList = new List<dynamic>();
    foreach(var node in iter)
    {
        outputList.Add(p(node.ToString()));
    }
    writer.Write(div(outputList));
}, "text/html");

In [1]:
root.Select("dataset/title", nsmgr)

## Using XSL Stylesheets

We can also perform XSL Transforms on the data to produce an HTML document using the XslCompiledTransform class as follows.  Here we are using a the stylesheet eml.xslt to retrieve the document title, abstract and citation information.

In [1]:
using System.Xml.Xsl;
using System.Xml;
using Microsoft.AspNetCore.Html;

using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

using(var stringWriter = new StringWriter())
using(var writer = new XmlTextWriter(stringWriter))
{
    XslCompiledTransform xslt = new XslCompiledTransform();
    xslt.Load("./data/eml.xslt");
    xslt.Transform(emlPath, writer);
    display(new HtmlString(stringWriter.ToString()));
}